# ISIC API Encapsulation

In [ ]:
import requests


class ISICApi(object):
    def __init__(self, hostname='https://isic-archive.com',
                 username=None, password=None):
        self.baseUrl = f'{hostname}/api/v1'
        self.authToken = None

        if username is not None:
            if password is None:
                password = input(f'Password for user "{username}":')
            self.authToken = self._login(username, password)

    def _makeUrl(self, endpoint):
        return f'{self.baseUrl}/{endpoint}'

    def _login(self, username, password):
        authResponse = requests.get(
            self._makeUrl('user/authentication'),
            auth=(username, password)
        )
        if not authResponse.ok:
            raise Exception(f'Login error: {authResponse.json()["message"]}')

        authToken = authResponse.json()['authToken']['token']
        return authToken

    def get(self, endpoint):
        url = self._makeUrl(endpoint)
        headers = {'Girder-Token': self.authToken} if self.authToken else None
        return requests.get(url, headers=headers)

    def getJson(self, endpoint):
        return self.get(endpoint).json()

    def getJsonList(self, endpoint):
        endpoint += '&' if '?' in endpoint else '?'
        LIMIT = 50
        offset = 0
        while True:
            resp = self.get(
                f'{endpoint}limit={LIMIT:d}&offset={offset:d}'
            ).json()
            if not resp:
                break
            for elem in resp:
                yield elem
            offset += LIMIT

# Connect to API

In [ ]:
api = ISICApi(username="spaniernathan", password="4YwB3z&f78BjdBp4W2ghBt^k8YT&HZDS")

# Choose the number of image

In [ ]:
import csv

datasetTotalSize = 200

benign_list = []
malignant_list = []

with open('metadata.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if len(benign_list) == datasetTotalSize/2 and len(malignant_list) == datasetTotalSize/2:
            break
        if (row['meta.clinical.benign_malignant'] == "benign") and len(benign_list) < datasetTotalSize/2:
            benign_list.append({'id': row['_id'], 'name': row['name'], 'class': row['meta.clinical.benign_malignant']})
        elif (row['meta.clinical.benign_malignant'] == "malignant") and len(malignant_list) < datasetTotalSize/2:
            malignant_list.append({'id': row['_id'], 'name': row['name'], 'class': row['meta.clinical.benign_malignant']})
        
imageFileResp = api.get('image/%s/download' % image['_id'])
    imageFileResp.raise_for_status()
    imageFileOutputPath = os.path.join(savePath, '%s.jpg' % image['name'])
    with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
        for chunk in imageFileResp:
            imageFileOutputStream.write(chunk)